In [1]:
from amuse.units import units
from amuse.lab import Huayno, nbody_system, new_galactics_model
from amuse.lab import Gadget2
from amuse.community.ph4.interface import ph4
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from amuse.couple import bridge
from amuse.community.hermite.interface import Hermite
from astropy.io import fits

In [2]:
%%time

# Milky Way := Mdisk = 4.5 Mbulge | Mhalo = 100 Mbulge
# Andromeda := Mdisk = 4-7 Mbulge | Mhalo = 87 Mbulge
# Mdisk = 5 Mbulge, Mhalo = 95 Mbulge

n_halo  = 100000
n_bulge = 50000
n_disk  = 50000
M_galaxy = 1e12 | units.MSun
R_galaxy = 80  | units.kpc
converter = nbody_system.nbody_to_si(M_galaxy, R_galaxy)

galaxy1 = new_galactics_model(n_halo,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge,
                                disk_number_of_particles=n_disk)
# Order of particles: Disk => Bulge => Halo

CPU times: user 5min 37s, sys: 8.43 s, total: 5min 45s
Wall time: 5min 51s


In [7]:
galaxy1[0].mass, galaxy1[n_bulge].mass, galaxy1[n_halo].mass

(quantity<8.60749885164e+35 kg>,
 quantity<4.73984840647e+35 kg>,
 quantity<1.92218326371e+37 kg>)

In [ ]:
%%time
M_galaxy_2 = 1e12 | units.MSun
R_galaxy_2 = 80  | units.kpc
converter_2 = nbody_system.nbody_to_si(M_galaxy_2, R_galaxy_2)
galaxy2 = new_galactics_model(n_halo,
                                converter_2,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge,
                                disk_number_of_particles=n_disk)

In [ ]:
converter = nbody_system.nbody_to_si(1.e12|units.MSun, 100|units.kpc)
dynamics = Gadget2(converter, number_of_workers=2)

In [ ]:
%%time
galaxy1.rotate(0., np.pi/2, 0.)
galaxy2.x  += 400 | units.kpc
galaxy2.vx += -100 |units.kms
galaxy2.vy += +40 |units.kms

dynamics.parameters.epsilon_squared = (100|units.parsec)**2
set1 = dynamics.particles.add_particles(galaxy1)
set2 = dynamics.particles.add_particles(galaxy2)
#dynamics.particles.move_to_center()

In [ ]:
channel = dynamics.particles.new_channel_to(dynamics.particles)

times = np.arange(0, 3501, 1) | units.Myr
threshold = 10 |units.Myr 

for time in tqdm(range(0, len(times))):
    dynamics.evolve_model(times[time])
    channel.copy()
    if times[time] %threshold == 0|units.Myr:
        plt.figure(figsize = (10,8))
        plt.scatter(set1.x[:int(n_disk)].value_in(units.kpc), set1.y[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set1.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set1.y[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.x[:int(n_disk)].value_in(units.kpc), set2.y[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set2.y[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        #plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-1000,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig("Tidal_Wave/Large/xy/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        plt.figure(figsize = (10,8))
        plt.scatter(set1.x[:int(n_disk)].value_in(units.kpc), set1.z[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set1.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set1.z[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.x[:int(n_disk)].value_in(units.kpc), set2.z[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set2.z[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        #plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("z [kpc]")
        plt.xlim(-1000,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig("Tidal_Wave/Large/xz/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        plt.figure(figsize = (10,8))
        plt.scatter(set1.y[:int(n_disk)].value_in(units.kpc), set1.z[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set1.y[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set1.z[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.y[:int(n_disk)].value_in(units.kpc), set2.z[:int(n_bulge)].value_in(units.kpc), s=0.3)
        plt.scatter(set2.y[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set2.z[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3)
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        #plt.legend()
        plt.xlabel("y [kpc]")
        plt.ylabel("z [kpc]")
        plt.xlim(-500,500)
        #plt.axis("equal")
        plt.ylim(-500,500)
        plt.savefig("Tidal_Wave/Large/yz/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        col21 = fits.Column(name='set_1_x', format='E', array=np.array(set1.x.value_in(units.m)))
        col22 = fits.Column(name='set_1_y', format='E', array=np.array(set1.y.value_in(units.m)))
        col23 = fits.Column(name='set_1_z', format='E', array=np.array(set1.z.value_in(units.m)))
        col24 = fits.Column(name='set_1_vx', format='E', array=np.array(set1.vx.value_in(units.kms)))
        col25 = fits.Column(name='set_1_vy', format='E', array=np.array(set1.vy.value_in(units.kms)))
        col26 = fits.Column(name='set_1_vz', format='E', array=np.array(set1.vz.value_in(units.kms)))

        
        col31 = fits.Column(name='set_2_x', format='E', array=np.array(set2.x.value_in(units.m)))
        col32 = fits.Column(name='set_2_y', format='E', array=np.array(set2.y.value_in(units.m)))
        col33 = fits.Column(name='set_2_z', format='E', array=np.array(set2.z.value_in(units.m)))
        col34 = fits.Column(name='set_2_vx', format='E', array=np.array(set2.vx.value_in(units.kms)))
        col35 = fits.Column(name='set_2_vy', format='E', array=np.array(set2.vy.value_in(units.kms)))
        col36 = fits.Column(name='set_2_vz', format='E', array=np.array(set2.vz.value_in(units.kms)))

        cols = fits.ColDefs([col21,col22,col23,col24,col25,col26,col31,col32,col33,col34,col35,col36])
        hdu = fits.BinTableHDU.from_columns(cols)
        name = "Tidal_Wave/Large/Data/Gadget2_save_data_%04d.fits"%time
        hdu.writeto(name,overwrite =True)
        
        
        
        
dynamics.stop()